In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./graphs

%tensorflow_version 2.x

!apt-get update -y
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install x11-utils
!apt install xvfb

!dpkg --configure -a
!pip install tqdm


#http://fnl.es/a-quick-reference-for-working-with-tensorflow.html

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]


KeyboardInterrupt: ignored

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
    #%tensorflow_version 1.xe
    
    if not os.path.exists('.setup_complete'):
        !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/spring20/setup_colab.sh -O- | bash
        !touch .setup_complete

    

In [ ]:
import time
import cv2
import numpy as np
from gym.core import Wrapper
from gym.spaces.box import Box

class PreprocessAtari(Wrapper):
  def __init__(self, env, height=42, width=42, color=False,
              crop=lambda img: img, n_frames=4, reward_scale=1):
    """A gym wrapper that reshapes, crops and scales image into the desired shapes"""
    super(PreprocessAtari, self).__init__(env)
    self.img_size = (height, width)
    self.crop = crop
    self.color = color

    self.reward_scale = reward_scale
    n_channels = (3 * n_frames) if color else n_frames

    obs_shape =  (height, width, n_channels)

    self.observation_space = Box(0.0, 1.0, obs_shape)
    self.framebuffer = np.zeros(obs_shape, 'float32')

  def reset(self):
    """Resets the game, returns initial frames"""
    self.framebuffer = np.zeros_like(self.framebuffer)
    self.update_buffer(self.env.reset()) # #Environment returns new_image, r, done, info
    return self.framebuffer

  def step(self, action):
    """Plays the game for 1 step, returns frame buffer"""
    new_img, r, done, info = self.env.step(action)
    self.update_buffer(new_img)

    return self.framebuffer, r * self.reward_scale, done, info

  ### image processing ###

  def update_buffer(self, img):
    img = self.preproc_image(img)
    offset = 3 if self.color else 1
    axis = -1
    cropped_framebuffer = self.framebuffer[:, :, :-offset]
    self.framebuffer = np.concatenate([img, cropped_framebuffer], axis=axis)

  def preproc_image(self, img):
    """what happens to the observation"""
    img = self.crop(img)
    img = cv2.resize(img / 255, self.img_size, interpolation=cv2.INTER_LINEAR)
    if not self.color:
        img = img.mean(-1, keepdims=True)
    return img


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import gym
from IPython import display as ipythondisplay
from IPython.display import clear_output
from pyvirtualdisplay import Display

import tensorflow.compat.v1 as tf
from collections import deque
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Input , Concatenate,Flatten,ConvLSTM2D,LSTM,Reshape , BatchNormalization,TimeDistributed
from tensorflow.keras.optimizers import Adam
import random
import tqdm
import pandas as pd

tf.disable_eager_execution()
from google.colab import drive
drive.mount('/content/drive')
import math

!pip install -q tf-agents
from tf_agents.utils import  value_ops

In [ ]:


def make_env():
  env = gym.make("KungFuMasterDeterministic-v0")
  env = PreprocessAtari(
      env, height=42, width=42,
      crop=lambda img: img[60:-30, 5:],
      color=False, n_frames=4)
  return env

env = make_env()

obs_shape = env.observation_space.shape
n_actions = env.action_space.n

print("Observation shape:", obs_shape)
print("Num actions:", n_actions)
print("Action names:", env.env.env.get_action_meanings())


In [ ]:
class Agent:
  def __init__(self, name, state_shape, n_actions,sess, isConvLSTM,  reuse=False):
    with tf.variable_scope(name, reuse=reuse):

      inputs = Input(shape=state_shape)
      x = Conv2D(32, (3, 3), strides=2, activation='relu')(inputs)
      x = Conv2D(32, (3, 3), strides=2, activation='relu')(x)
      x = Conv2D(32, (3, 3), strides=2, activation='relu')(x)
      x = Flatten()(x)
      if (isConvLSTM == False):
        x = Dense(128, activation='relu')(x)
      else:
        rnn_in = Reshape((1 ,x.shape[1]), input_shape=x.shape)(x)
        x = LSTM(128, return_sequences=False, name ="lstm_enc_1")(rnn_in)

      logits = Dense(n_actions, activation='linear')(x)
      state_value = Dense(1, activation='linear')(x)

      self.network = Model(inputs=inputs, outputs=[logits, state_value])

      # prepare a graph for agent step
      self.state_t = tf.placeholder('float32', [None,] + list(state_shape))
      self.agent_outputs = self.symbolic_step(self.state_t)
    
  def symbolic_step(self, state_t):
    l,s, = self.network(state_t)

    logits = l
    state_values = s[:,0]

    return logits, state_values
  
  def sample_actions(self, state_t):
    """pick actions given numeric agent outputs (np arrays)"""
    
    logits,_= sess.run(self.agent_outputs, {self.state_t: state_t})
    policy = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    selected_action =np.array([np.random.choice(len(p), p=p) for p in policy])
    return selected_action


In [ ]:
class KungFuMaster:
  def __init__(self, sess, env, agent, lstm,gae):
    
    self.filepath = "/content/drive/My Drive/Colab Notebooks/RL/KungFuMasterDeterministic/data.txt"
    self.batch_size =10
    self.memory = deque(maxlen= self.batch_size)
    self.nEnviroment=10
    self.Training= 200000
    self.learning_rate= 1e-4
    self.entropy_factor = 0.001

    self.rewards_history = []
    self.entropy_history = []
    self.actor_history =[]
    self.critic_history =[]
    self.advantage_history = []
    self.target_history =[]
    self.v_history =[]
    self.actionList =[]
    self.game_reward = 0
    
    self.is_gae = gae
    self.isConvLSTM = lstm 
    self.env = env
    self.obs_shape = self.env.observation_space.shape
    self.n_actions = self.env.action_space.n

    print("Observation shape:", self.obs_shape)
    print("Num actions:", self.n_actions)
    self.reward_step =0
    self.writer = tf.summary.FileWriter('./graphs', sess.graph)

    self.agent = agent
    self.states_ph = tf.placeholder('float32', [None,] + list(self.obs_shape))    
    self.next_states_ph = tf.placeholder('float32', [None,] + list(self.obs_shape))
    self.actions_ph = tf.placeholder('int32', (None,))
    self.rewards_ph = tf.placeholder('float32', (None,))
    self.is_done_ph = tf.placeholder('float32', (None,))

    # logits[n_envs, n_actions] and state_values[n_envs, n_actions]
    self.logits, self.state_values = self.agent.symbolic_step(self.states_ph)
    self.next_logits, self.next_state_values = self.agent.symbolic_step(self.next_states_ph)

    # There is no next state if the episode is done!
    self.next_state_values = self.next_state_values * (1 - self.is_done_ph)

    # probabilities and log-probabilities for all actions
    self.probs = tf.nn.softmax(self.logits, axis=-1)            # [n_envs, n_actions]
    self.logprobs = tf.nn.log_softmax(self.logits, axis=-1)     # [n_envs, n_actions]

    # log-probabilities only for agent's chosen actions
    self.logp_actions = tf.reduce_sum(self.logprobs * tf.one_hot(self.actions_ph, self.n_actions), axis=-1) # [n_envs,]

    self.gamma = 0.99
    self.advantage = self.rewards_ph + self.gamma*self.next_state_values - self.state_values
    self.entropy = -tf.reduce_sum(self.probs * self.logprobs, 1, name="entropy")
    self.target_state_values = self.rewards_ph+self.gamma*self.next_state_values

    self.actor_loss = -tf.reduce_mean(self.logp_actions * tf.stop_gradient(self.advantage), axis=0) - self.entropy_factor * tf.reduce_mean(self.entropy, axis=0)
    self.critic_loss = tf.reduce_mean((self.state_values - tf.stop_gradient(self.target_state_values))**2, axis=0)
    self.train_step = tf.train.AdamOptimizer(self.learning_rate).minimize(self.actor_loss + self.critic_loss)

    self.summary_adv = tf.summary.scalar('advantage_', tf.reduce_mean(self.advantage, axis=-1))
    self.summary_ent = tf.summary.scalar('entropy_',tf.reduce_mean(self.entropy, axis=-1))
    self.summary_target = tf.summary.scalar('target_state_values_',tf.reduce_mean(self.target_state_values, axis=-1))
    self.summary_act_loss = tf.summary.scalar('actor_loss_', self.actor_loss)
    self.summary_crit_loss = tf.summary.scalar('critic_loss_', self.critic_loss)
    self.summary_batchreward = tf.summary.scalar('batchreward', tf.reduce_mean(self.rewards_ph, axis=-1))
    self.summary_v = tf.summary.scalar('state_values', tf.reduce_mean(self.state_values, axis=-1))
    self.summary_r = tf.summary.scalar('mean_reward', np.mean(self.game_reward))

  def evaluate(self, n_games=1):
      self.game_rewards = []
      for _ in range(n_games):
          state = self.env.reset()

          total_reward = 0
          while True:
              action = self.agent.sample_actions([state])[0]
              state, reward, done, info = self.env.step(action)
              total_reward += reward
              if done:
                  break

          self.game_rewards.append(total_reward)
      
      self.game_reward = np.mean(self.game_rewards)
      self.writer.add_summary(sess.run(self.summary_r),self.reward_step)
      self.reward_step +=1

      return self.game_rewards


  def train(self,batch_states,batch_actions,batch_next_states,batch_rewards,batch_done,step):

    feed_dict = {
            self.states_ph: batch_states,
            self.actions_ph: batch_actions,
            self.next_states_ph: batch_next_states,
            self.rewards_ph: batch_rewards,
            self.is_done_ph: batch_done,
        }

    _, ent_t,act,crit,adv,targ,v,sum_adv,sum_ent,sum_target,sum_act_loss,sum_crit_loss,sum_batchreward,sum_v = sess.run([self.train_step, self.entropy,self.actor_loss ,self.critic_loss,self.advantage,
                                             self.target_state_values,self.state_values,
                                             self.summary_adv,self.summary_ent,self.summary_target,self.summary_act_loss,self.summary_crit_loss,
                                             self.summary_batchreward,self.summary_v], feed_dict)

    self.writer.add_summary(sum_adv, step)
    self.writer.add_summary(sum_ent, step)
    self.writer.add_summary(sum_target, step)
    self.writer.add_summary(sum_act_loss, step)
    self.writer.add_summary(sum_crit_loss, step)
    self.writer.add_summary(sum_batchreward, step)
    self.writer.add_summary(sum_v, step)
      
      
    self.entropy_history.append(np.mean(ent_t))
    self.actor_history.append(np.mean(act))
    self.critic_history.append(np.mean(crit))
    self.advantage_history.append(np.mean(adv))
    self.target_history.append(np.mean(targ))
    self.v_history.append(np.mean(v))
    for a in batch_actions:
      self.actionList.append(a)

  def ewma(self,x, span=100):
    return pd.DataFrame({'x':np.asarray(x)}).x.ewm(span=span).mean().values

  def plots(self):
    ipythondisplay.clear_output(wait=True)
    plt.figure(figsize=[35,13])
    plt.subplot(2,4,1)
    plt.plot(self.rewards_history, label='reward')
    plt.plot(self.ewma(np.array(self.rewards_history), span=10), marker='.', label='rewards ewma@10')
    plt.title("Session rewards"); plt.grid(); plt.legend()

    plt.subplot(2,4,2)
    plt.plot(self.actor_history, label='Actor Loss')
    plt.plot(self.ewma(np.array(self.actor_history), span=10), marker='.', label='Actor Loss ewma@10')
    plt.title("Policy Function"); plt.grid(); plt.legend()  

    plt.subplot(2,4,3)
    plt.plot(self.entropy_history, label='Entropy')
    plt.plot(self.ewma(np.array(self.entropy_history), span=10), marker='.', label='Entropy ewma@10')
    plt.title("Policy Function"); plt.grid(); plt.legend()      
    
    plt.subplot(2,4,4)
    plt.plot(self.target_history, label='Target ')
    plt.plot(self.v_history, label='V')
    plt.title("Target/V"); plt.grid(); plt.legend()

    plt.subplot(2,4,5)
    plt.plot(self.advantage_history, label='Advantage')
    plt.title("Advantage"); plt.grid(); plt.legend()  

    plt.subplot(2,4,6)
    plt.plot(self.critic_history, label='Critic Loss')
    plt.plot(self.ewma(np.array(self.critic_history), span=10), marker='.', label='Critic Loss ewma@10')
    plt.title("Value function"); plt.grid(); plt.legend()   

    plt.subplot(2,4,7)
    plt.plot(self.actionList, label='Actions')
    plt.title("Actions"); plt.grid(); plt.legend()  

    plt.show()
    


In [ ]:
class EnvBatch:
  def __init__(self, n_envs = 10):
    """ Creates n_envs environments and babysits them for ya' """
    self.envs = [make_env() for _ in range(n_envs)]
    
  def reset(self):
    """ Reset all games and return [n_envs, *obs_shape] observations """
    return np.array([env.reset() for env in self.envs])
  
  def step(self, actions):
    """
    Send a vector[batch_size] of actions into respective environments
    :returns: observations[n_envs, *obs_shape], rewards[n_envs], done[n_envs,], info[n_envs]
    """

    results = [env.step(a) for env, a in zip(self.envs, actions)]
    new_obs, rewards, done, infos = map(np.array, zip(*results))
    
    # reset environments automatically
    for i in range(len(self.envs)):
        if done[i]:
            new_obs[i] = self.envs[i].reset()
    
    return new_obs, rewards, done, infos

In [ ]:
sess = tf.Session(graph= tf.get_default_graph())
RESET = 1
is_gae = False
isConvLSTM = True 

In [ ]:
display = Display(visible=0, size=(400, 300))
display.start()
agent = Agent("network",env.observation_space.shape, env.action_space.n,sess, isConvLSTM)
kungFu = KungFuMaster(sess,env,agent, isConvLSTM,is_gae)

In [ ]:

checkpoint_directory = "/content/drive/My Drive/Colab Notebooks/RL/KungFuMasterDeterministic/"

if (RESET == 1):
  # ************ RESET everything **************************
  f=open(kungFu.filepath, "w")
  f.write("")
  f.close()
  sess.run(tf.global_variables_initializer())
  saver = tf.train.Saver()

else:
  # ************ LOAD everything **************************
  saver = tf.train.Saver()
  saver.restore(sess, tf.train.latest_checkpoint(checkpoint_directory))

  f=open(kungFu.filepath,'r')
  for line in f:
    kungFu.rewards_history.append(float(line.split()[0]))
    kungFu.actor_history.append(float(line.split()[1]))
    kungFu.entropy_history.append(float(line.split()[2]))
    kungFu.target_history.append(float(line.split()[3]))
    kungFu.v_history.append(float(line.split()[4]))
    kungFu.advantage_history.append(float(line.split()[5]))
    kungFu.critic_history.append(float(line.split()[6]))
  f.close()
  kungFu.plots()


#https://itnext.io/how-to-use-tensorboard-5d82f8654496

In [ ]:
# Validate
rewards = kungFu.evaluate(n_games=3)
print(rewards)

env_batch = EnvBatch(10)
batch_states = env_batch.reset()

batch_actions = kungFu.agent.sample_actions(batch_states)
batch_next_states, batch_rewards, batch_done, _ = env_batch.step(batch_actions)

print("State shape:", batch_states.shape)
print("Actions:", batch_actions)
print("Rewards:", batch_rewards)
print("Done:", batch_done)

#Initialize
env_batch = EnvBatch(10)
batch_states = env_batch.reset()

In [ ]:
  %tensorboard --logdir graphs
  time.sleep(10)

In [ ]:
f=open(kungFu.filepath, "w")
with tqdm.trange(len(kungFu.entropy_history), kungFu.Training) as t:
  for i in t:
    batch_actions = kungFu.agent.sample_actions(batch_states)
    batch_next_states, batch_rewards, batch_done, _ = env_batch.step(batch_actions)

    batch_rewards = batch_rewards * 0.01
    kungFu.train(batch_states,batch_actions,batch_next_states,batch_rewards,batch_done,i)
    batch_states = batch_next_states
    
    if i % 500 == 0:

      if i % 2500 == 0:
        saver.save(sess,checkpoint_directory)
        kungFu.rewards_history.append(np.mean(kungFu.evaluate(n_games=3)))
        if kungFu.rewards_history[-1] >= 5000:
          print("Your agent has earned the yellow belt")

        f.write("%f %f %f %f %f %f %f\n" % (kungFu.rewards_history[-1],kungFu.actor_history[-1], kungFu.entropy_history[-1],
                                          kungFu.target_history[-1],kungFu.v_history[-1],kungFu.advantage_history[-1],kungFu.critic_history[-1]))
        
        if len(kungFu.rewards_history)> 100 :
          if np.mean(kungFu.rewards_history[-100:]) >= 200:
            print("Your agent reached the objetive")
            break

      if i %10000 ==0:
        f.close()
        f=open(kungFu.filepath, "a+")
      kungFu.plots()
      
f.close()

In [ ]:
#https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
#https://github.com/tensorflow/tensorboard/issues/3186  -  went to chrome://settings/content/cookies and had to unable the "block indirect cookies" setting

In [ ]:
#https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/hyperparameter_tuning_with_hparams.ipynb#scrollTo=mVtYvbbIWRkV


In [ ]:
display = Display(visible=0, size=(400, 300))
display.start()


def evaluate2(agent, env, n_games=1):
  """Plays an a game from start till done, returns per-game rewards """
  game_rewards = []
  for _ in range(n_games):
      state = env.reset()

      total_reward = 0
      while True:
          action = agent.sample_actions([state])[0]
          state, reward, done, info = env.step(action)
          total_reward += reward
          if done:
              break

      game_rewards.append(total_reward)
  return game_rewards

env_monitor = gym.wrappers.Monitor(env, directory="/content/drive/My Drive/Colab Notebooks/RL/KungFuMasterDeterministic/videos", force=True)
final_rewards = evaluate2(agent, env_monitor, n_games=10)
env_monitor.close()

print("Final mean reward:", np.mean(final_rewards))